In [1]:
import acquire
import outliers
import prep
import pandas as pd
import numpy as np
import math
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_squared_error


import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

from sklearn import metrics

from scipy.stats import entropy

import matplotlib.dates as mdates

## Questions To Answer
- Which lesson appears to attract the most traffic consistently across cohorts (per program)?
- Is there a cohort that referred to a lesson significantly more that other cohorts seemed to gloss over? 
- Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students? 
- Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses? Any odd user-agents? 
- At some point in the last year, ability for students and alumni to cross-access curriculum (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before? 
- What topics are grads continuing to reference after graduation and into their jobs (for each program)? 
- Which lessons are least accessed? 
- Anything else I should be aware of? 

In [2]:
df = prep.prep_curriculum_data()

In [3]:
df

,page_viewed,user_id,cohort_id,ip,name,start_date,end_date,program_id,conv_ip
2018-01-26 09:56:02,java-ii,1.0,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,971051961.0
2018-01-26 09:56:05,java-ii/object-oriented-programming,1.0,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,971051961.0
2018-01-26 09:56:06,slides/object_oriented_programming,1.0,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,971051961.0
2018-01-26 10:40:15,javascript-i/functions,1.0,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,971051961.0
2018-01-26 11:26:13,java-i,1.0,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0,971051961.0
...,...,...,...,...,...,...,...,...,...
NaT,NaN,NaN,10.0,NaN,Balboa,2015-11-03,2016-03-11,4.0,NaN
NaT,NaN,NaN,20.0,NaN,Redwood,2017-07-31,2017-12-06,2.0,NaN
NaT,NaN,NaN,30.0,NaN,Ada,2019-02-04,2019-06-16,3.0,NaN
NaT,NaN,NaN,60.0,NaN,Io,2020-07-20,2021-01-21,2.0,NaN


In [4]:
df.isnull().sum()

page_viewed    7
user_id        6
cohort_id      0
ip             6
name           0
start_date     0
end_date       0
program_id     0
conv_ip        6
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.index.name = 'datetime'

In [7]:
pd.options.display.float_format = '{:.1f}'.format
ip_outliers = outliers.emp_rule(df.conv_ip, 3)
ip_outliers.x.value_counts()

192171117210.0    8062
173173115201.0    1157
173174211206.0     945
170248173245.0     855
170248173247.0     822
                  ... 
199116118166.0       1
173174147198.0       1
173239232157.0       1
173127108144.0       1
192241235228.0       1
Name: x, Length: 365, dtype: int64

In [ ]:
user_outliers = outliers.emp_rule(df.user_id, 2)
user_outliers.x.value_counts()

### Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
plt.figure(figsize=(8,5))
wd = df[df.program_id != 3]
wd = wd[wd.name != 'Staff']
wd = wd[wd.name != 'Unknown']
wd.page_viewed.value_counts().plot()
plt.xticks(rotation=90)
plt.ylabel('Number of Requests')
plt.xlabel('Curriculum Accessed')
plt.title('Subject Accessed The Most By Web Dev Students')

In [ ]:
plt.figure(figsize=(8,5))
ds = df[df.program_id == 3]
ds.page_viewed.value_counts().plot()
plt.xticks(rotation=90)
plt.ylabel('Number of Requests')
plt.xlabel('Curriculum Accessed')
plt.title('Subject Accessed The Most By Data Science Students')

## Conclusion
- Web Dev Students access the Javascript I portion of the curriculum the most.
- Data Science students access Intro to Data science the most.

### When were the programs accessing these curriculum subjects?

In [ ]:
plt.figure(figsize=(8,5))
wd[wd.page_viewed == 'javascript-i'].page_viewed.resample('w').count().plot()
plt.title('Javascript I Weekly Page Visits')
plt.xlabel('Weeks')
plt.ylabel('Number of Requests')

In [ ]:
plt.figure(figsize=(8,5))
ds[ds.page_viewed == '1-fundamentals/1.1-intro-to-data-science'].page_viewed.resample('w').count().plot()
plt.title('Intro To Data Science Weekly Page Visits')
plt.xlabel('Weeks')
plt.ylabel('Number of Requests')

### Conclusions
- Web Dev students appear to have been constantly referring back to the Javascript I material
- Data Scientist accessed the Intro to Data Science lesson in bulk and a lot more often for a short period of time.

### Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [ ]:
wd.groupby(by = wd.name).page_viewed.value_counts().nlargest(10)

In [ ]:
ds.groupby(by = ds.name).page_viewed.value_counts().nlargest(10)

### Conclusions

Web Dev:

- Ceres spent a significant amount of time on index.html compared to other cohorts.
- It appears the vast majority spent their time on Java related subjects.


Data Science:

- Darden accessed the Classification Overview lesson significantly the most.
- Curies spent more time on the SQL Overview than other cohorts.

### Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [ ]:
active_students = df[(df.name == 'Jupiter') | (df.name == 'Bash') | (df.name == 'Darden') | (df.name == 'Hyperion')| (df.name == 'Io')]

In [ ]:
active_students.head()

In [ ]:
active_students.user_id.value_counts().tail(20)

In [ ]:
barely_active = df[(df.user_id == 679.0)|(df.user_id == 697.0)|(df.user_id == 772.0)|(df.user_id == 785.0)|(df.user_id == 780.0)|(df.user_id == 783.0)]

In [ ]:
barely_active

In [ ]:
df[df.ip == '136.50.70.27'].user_id.value_counts()

In [ ]:
df[df.ip == '99.132.128.255'].user_id.value_counts()

### Conclusions

- Web Dev and Data Science Cohorts both had roughly the same trend as to student inactivity.
- The lowest activity on the curriculum came out of both Darden and Jupiter which are the most recent classes at CodeUp.
- The major similarity between the two cohorts and lack of activity is that both cohorts were full time at home.
- Zoom appears to have had a major impact on the activity to the curriculum from past cohorts, since the instructors have the ability to screen share the curriculum.
- It appears students are just referring to the curriculum to read and study, rather than follow along in class as it was before.
- One big factor is it appears that there is a new user id generated for every device a student accesses the curriculum.
- Shown above, a student will appear to be not interacting but they are actually more active with another user id.

### Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses? Any odd user-agents?

In [ ]:
sus_users = df[df.name == 'Unknown']
sus_users.head(),

In [ ]:
sus_users.ip.value_counts()

In [ ]:
df[df.ip == '97.105.19.58'].user_id.value_counts()

In [ ]:
df[df.ip == '70.117.16.60'].user_id.value_counts()

In [ ]:
df[df.ip == '70.117.16.60'].head()

In [ ]:
df[df.ip == '70.94.165.107'].user_id.value_counts()

In [ ]:
df[df.ip == '70.94.165.107'].head()

In [ ]:
df[df.ip == '23.116.170.48'].user_id.value_counts()

In [ ]:
df[df.ip == '23.116.170.48'].head()

In [ ]:
df[df.name == 'Hyperion'].user_id.value_counts()

In [ ]:
df[df.name == 'Darden'].user_id.value_counts()

In [ ]:
anom = outliers.emp_rule(df.conv_ip, 5)
anom.x.value_counts()

In [ ]:
df[df.conv_ip == 216177181227.0].user_id.value_counts()

In [ ]:
df[df.conv_ip == 216177181227.0].head()

In [ ]:
df[df.conv_ip == 216177165194.0].user_id.value_counts()

In [ ]:
df[df.conv_ip == 216177165194.0].head()

In [ ]:
df[df.conv_ip == 209107196134.0].user_id.value_counts()

In [ ]:
df[df.conv_ip == 209107196134.0]

In [ ]:
df[df.conv_ip == 209205120222.0].user_id.value_counts()

In [ ]:
df[df.conv_ip == 209205120222.0]

In [ ]:
df[df.conv_ip == 209205221162.0].user_id.value_counts()

In [ ]:
df[df.conv_ip == 209205221162.0]

### Conclusions

IT Event?:

- It appears that something out of the norm happened with Hyperion's Cohort that caused every student to be reassigned a new user id and not registered to a specific cohort, program, or dates attended.
- This event occured on or around July 20th.
- It seems that all the students had continued on their designated curriculum path but under new untracked user id's.


Anomolies:

- The IP 216.177.165.194 is something I would flag as an alert, as they were an Ada student that appears to have completed the program, but utilizing the intro modules after graduation. Unknown if it was for refresher purposes or providing material to someone as a trial.

- The IP 209.107.196.134 accessed the curriculum for Web Dev 3 years after the listed completion date from Atlanta, Georgia, however, it appears the only information they were interested in was the curriculum related to using GitHub. May not be any sort of breach.

- The rest of the IP's appear to just be outliers and we can ignore the noise that is most likely resulting from multiple IP use.

### At some point in the last year, ability for students and alumni to cross-access curriculum (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

In [ ]:
this_years_cohorts = df[df.start_date > '2020-01-01']

In [ ]:
this_years_cohorts.groupby(this_years_cohorts.cohort_id).page_viewed.value_counts()

In [ ]:
df[df.cohort_id == 55.0].head()

In [ ]:
currie = df[df.cohort_id == 55.0]

In [ ]:
currie[currie.page_viewed == 'java-ii/object-oriented-programming']

In [ ]:
df[df.cohort_id == 61.0].head()

In [ ]:
bash = df[df.cohort_id == 61.0]

In [ ]:
bash[bash.page_viewed == 'anomaly-detection/continuous-probabilistic-methods']

In [ ]:
bash[bash.user_id == 730.0].head()

### Conclusions
- It appears that the end of cross access was not successful and individuals are still able to access between both programs.
- The bash student with user id 730 has been going back and forth between both programs curriculums as recently as a few weeks ago
- The only thing I could assume is it could potentially be a student that became a staff member or instructor and are accessing on their own.
- The IP belongs to someone in Dallas, TX so it's possible it could be a Dallas Codeup Office member.
- Overall, if the goal was to disable students from cross access, we would have to assume that it did not work based on this data.
- Further investigation would be needed to determine if this was a special situation.

### What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [ ]:
wd_grads = df[df.end_date < '2019-01-01']

In [ ]:
wd_grads = wd_grads[wd_grads.name != 'Staff']

In [ ]:
wd_grads.head()

In [ ]:
wd_grads.tail()

In [ ]:
wd_grads.groupby(wd_grads.name).page_viewed.value_counts().nlargest(25)

In [ ]:
voyageurs = wd[wd.name == 'Voyageurs']

plt.figure(figsize=(8,5))
voyageurs[voyageurs.page_viewed == 'javascript-i'].page_viewed.resample('m').count().plot()
plt.title('Monthly Post Grad Voyageur Javascript I Traffic')
plt.xlabel('Months')
plt.ylabel('Number of Requests')

In [ ]:
wd_grads[wd_grads.name == 'Voyageurs'].tail()

In [ ]:
ds_grads = df[df.end_date > '2019-06-16']
ds_grads = ds_grads[ds_grads.end_date < '2020-07-07']
ds_grads = ds_grads[ds_grads.program_id == 3.0]

In [ ]:
ds_grads.head()

In [ ]:
ds_grads.tail()

In [ ]:
ds_grads.groupby(ds_grads.name).page_viewed.value_counts().nlargest()

In [ ]:
bayes = ds_grads[ds_grads.index > '2020-01-30']
bayes.head()

In [ ]:
plt.figure(figsize=(8,5))
bayes[bayes.page_viewed == '1-fundamentals/1.1-intro-to-data-science'].page_viewed.resample('m').count().plot()
plt.title('Monthly Post Grad Bayes Intro To Data Science Traffic')
plt.xlabel('Months')
plt.ylabel('Number of Requests')

In [ ]:
bayes.page_viewed.value_counts()

### Conclusions

Web Dev:

- Based on the graduated cohorts that have had ample time to get a job and be secure in their job, it appears that Java I - III and MySQL are the subjects in the curriculum being accessed the most.

- I did find it odd that the cohorts that have been accessing the curriculum the most is Voyageours and Teddy

- Voyageurs graduated 2 years ago and this much activity was somewhat odd as it's been as recent as last month.

Data Science:

- Since Ada and Bayes are the only cohorts that have had ample time to get jobs and be established in their roles, I went ahead and assessed these cohorts further.

- Ada's data is missing altogether, but based on Bayes, it appears that intro to data science and regression have been accessed the most.

- We can assume the access to the intro to data science was just as a means to attempt to recruit new students, so regression is a place we can focus that students appear to need a refresher on.

- Anomaly detection is the second highest if we ignore the intro portion of the curriculum.

### Which lessons are least accessed?

In [ ]:
cohorts_with_data = df[df.start_date > '2018-01-01']

In [ ]:
cohorts_with_data.head()

In [ ]:
split_lesson = cohorts_with_data['page_viewed'].str.split("/", n = 1, expand = True) 
split_lesson["lesson"]= split_lesson[0] 
split_lesson["subject"]= split_lesson[1] 

In [ ]:
split_lesson.drop(columns =[0,1], inplace = True)

In [ ]:
split_lesson.head()

In [ ]:
least_lesson = pd.concat([split_lesson, cohorts_with_data], axis = 1)

In [ ]:
least_lesson.head()

In [ ]:
least_lesson.groupby(least_lesson.name).lesson.value_counts().nsmallest()

In [ ]:
least_lesson = least_lesson[least_lesson.name != 'Andromeda']

In [ ]:
plt.figure(figsize=(8,5))
least_lesson[least_lesson.page_viewed == 'sessions-and-cookies'].page_viewed.resample('q').count().plot()
plt.title('Quarterly Sessions and Cookies Page Visits')
plt.xlabel('Quarters')
plt.ylabel('Number of Requests')

In [ ]:
plt.figure(figsize=(8,5))
least_lesson[least_lesson.page_viewed == 'working-with-time-series-data'].page_viewed.resample('d').count().plot()
plt.title('Daily Working With Time Series Data Page Visits')
plt.xlabel('Days')
plt.ylabel('Number of Requests')

In [ ]:
least_lesson.groupby(least_lesson.lesson).program_id.value_counts().nlargest(100)

### Conclusions

- I chose to focus on the more lessons that were more reasonably lacking access and traffic by assessing only the values that are related to each cohort and reasonable for lack of access.

- It's not logical that only 1 person would access the curriculum so I focused on grouping the values by lessons and expanding the data on the program id.


Web Dev:

- The least accessed lesson in the curriculum for Web Dev was Sessions and Cookies.

- This may have been due to the subject itself may not have been as complex, but that is to be speculated as I am not familiar with web development.


Data Science:

- The least accessed lesson in the curriculum for Data Science was Working With Time Series.

- This may be due to the part of the curriculum where you are learning more so from the instructors due to it's difficulty rather than trying to learn it on your own.

# Final Thoughts

- This data presented some opportunities to possibly review any gaps in potential curriculum theft.
- There are several points where students are accessing the curriculum many years after they have left Codeup.
- This leaves the potential for someone to share the curriculum with others, share with another organization, or saved the curriculum for themselves.
- A better security protocol for accessing the curriculum may benefit Codeup in the future as they continue to grow and differentiate themselves from other schools.
- There are also many accesses from untracked ip's and we have no idea if it's an actual student or admin.
- It also appears that there are some lessons that are overlapped by instructor tutorials so the students aren't referring directly to the curriculum for any further development.
- The crossover between access of both programs appears to have still kept going, but we would need further review to see if the Baye's student became an instructor or staff member and is just accessing both as part of the benefit of being an employee.
- Overall, this data is very useful for anomaly detection and discovery to see if there are any trends going on in each cohort.